In [15]:

def load_doc(filename):
    file = open(filename, 'r',encoding='utf-8')
    text = file.read()
    file.close()
    return text

In [16]:
in_filename = 'adam.txt'
doc = load_doc(in_filename)
print(doc[:200])

﻿Chapter I
The Workshop


With a single drop of ink for a mirror, the Egyptian sorcerer
undertakes to reveal to any chance comer far-reaching visions of the
past. This is what I undertake to do for yo


In [17]:
import string

def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [18]:
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['i', 'the', 'workshop', 'with', 'a', 'single', 'drop', 'of', 'ink', 'for', 'a', 'mirror', 'the', 'egyptian', 'sorcerer', 'undertakes', 'to', 'reveal', 'to', 'any', 'chance', 'comer', 'farreaching', 'visions', 'of', 'the', 'past', 'this', 'is', 'what', 'i', 'undertake', 'to', 'do', 'for', 'you', 'reader', 'with', 'this', 'drop', 'of', 'ink', 'at', 'the', 'end', 'of', 'my', 'pen', 'i', 'will', 'show', 'you', 'the', 'roomy', 'workshop', 'of', 'mr', 'jonathan', 'burge', 'carpenter', 'and', 'builder', 'in', 'the', 'village', 'of', 'hayslope', 'as', 'it', 'appeared', 'on', 'the', 'eighteenth', 'of', 'june', 'in', 'the', 'year', 'of', 'our', 'lord', 'the', 'afternoon', 'sun', 'was', 'warm', 'on', 'the', 'five', 'workmen', 'there', 'busy', 'upon', 'doors', 'and', 'windowframes', 'and', 'wainscoting', 'a', 'scent', 'of', 'pinewood', 'from', 'a', 'tentlike', 'pile', 'of', 'planks', 'outside', 'the', 'open', 'door', 'mingled', 'itself', 'with', 'the', 'scent', 'of', 'the', 'elderbushes', 'which'

In [19]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 198685


In [22]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding='utf-8')
    file.write(data)
    file.close()

In [23]:
out_filename = 'adam_sequences.txt'
save_doc(sequences, out_filename)

# Training the model

In [24]:
in_filename = 'adam_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [25]:
from keras.preprocessing.text import Tokenizer

# Encdiing the textual data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [26]:
vocab_size = len(tokenizer.word_index) + 1

In [27]:
import numpy as np
from keras.utils import to_categorical

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential()
model.add(layers.Embedding(vocab_size, 50, input_length=seq_length))
model.add(layers.LSTM(100, return_sequences=True))
model.add(layers.LSTM(100))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            571700    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 11434)             1154834   
                                                                 
Total params: 1,877,434
Trainable params: 1,877,434
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50
1553/1553 [==============================] - 273s 173ms/step - loss: 3.0235 - accuracy: 0.3415
Epoch 2/50
1553/1553 [==============================] - 288s 185ms/step - loss: 2.9873 - accuracy: 0.3459
Epoch 3/50
1553/1553 [==============================] - 294s 189ms/step - loss: 2.9775 - accuracy: 0.3483
Epoch 4/50
1553/1553 [==============================] - 298s 192ms/step - loss: 2.9584 - accuracy: 0.3511
Epoch 5/50
1553/1553 [==============================] - 298s 192ms/step - loss: 2.9470 - accuracy: 0.3529
Epoch 6/50
1553/1553 [==============================] - 304s 196ms/step - loss: 2.9374 - accuracy: 0.3545
Epoch 7/50
1553/1553 [==============================] - 308s 198ms/step - loss: 2.9230 - accuracy: 0.3577
Epoch 8/50
1553/1553 [==============================] - 307s 198ms/step - loss: 2.9122 - accuracy: 0.3584
Epoch 9/50
1553/1553 [==============================] - 309s 199ms/step - loss: 2.9051 - accuracy: 0.3598
Epoch 10/50
1553/1553 [=======================

In [35]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model.save('model_adam2.h5')
dump(tokenizer, open('tokenizer_adam2.pkl', 'wb'))